# Bibliotecas

In [25]:
pip install youtube_transcript_api fuzzywuzzy python-Levenshtein SpeechRecognition moviepy

In [26]:
from youtube_transcript_api import YouTubeTranscriptApi
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
import os
import speech_recognition as sr 
import moviepy.editor as mp
from google.colab import drive

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [27]:
from nltk.corpus import *

# Codigo



## YoutubeTrascript

In [ ]:
video_id = "z9rg82kBDik"
subtitles = YouTubeTranscriptApi.get_transcript(video_id, languages=['pt', 'en'])

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')
for i in range(len(subtitles)):
  text = nltk.word_tokenize(subtitles[i]['text'], language='portuguese')
  query = 'Trabalho'
  #query = 'KineckGame'
  #print(process.extract(query, nltk.pos_tag(text)))
  print(process.extractOne(query, nltk.pos_tag(text)))
  #print(process.extractOne(query, nltk.pos_tag(text), score_cutoff=50))
  if i == 8:
    break
  

(('trabalhar', 'NN'), 79)
(('gravar', 'NN'), 34)
(('certo', 'NN'), 24)
(('também', 'NN'), 36)
(('trabalha', 'NN'), 79)
(('trabalhar', 'NN'), 79)
(('programação', 'NN'), 45)
(('interativo', 'NN'), 45)
(('material', 'NN'), 45)


## Google Drive



In [31]:
drive.mount('/content/drive/')

Mounted at /content/drive/


### Extraindo Video

In [ ]:
video_drive = '/content/drive/My Drive/kineckgame.mp4'

In [ ]:
clip = mp.VideoFileClip(video_drive)
clip.audio.write_audiofile(r'converted.wav')

[MoviePy] Writing audio in converted.wav


100%|██████████| 10248/10248 [00:04<00:00, 2386.17it/s]

[MoviePy] Done.


In [ ]:
r = sr.Recognizer()
audio = sr.AudioFile("converted.wav")
with audio as source:
  audio_file = r.record(source)
result = r.recognize_google(audio_file)

RequestError: ignored

In [ ]:
# exporting the result 
with open('recognized.txt',mode ='w') as file: 
   file.write("Recognized Speech:") 
   file.write("\n") 
   file.write(result) 
   print("ready!")

### Extraindo Emenda

#### Extraindo Texto da Emenda do Drive

In [168]:
with open('/content/drive/My Drive/trabalhosIA/EmendaIA.txt', 'r') as f:
   emenda_drive_ia = f.read()
emenda_drive_ia

'Introdução à inteligência artificial: histórico, conceitos básicos, áreas de aplicação, tendências. Métodos de resolução de problemas. Técnicas de busca: com e sem informação (heurística e meta-heurística – Algoritmos Genéticos). Aprendizado de máquina. Representação do conhecimento. Redes Neurais.'

#### Organizar um texto de ementa em períodos

In [172]:
periodos = nltk.tokenize.sent_tokenize(emenda_drive_ia, language='portuguese')
periodos

['Introdução à inteligência artificial: histórico, conceitos básicos, áreas de aplicação, tendências.',
 'Métodos de resolução de problemas.',
 'Técnicas de busca: com e sem informação (heurística e meta-heurística – Algoritmos Genéticos).',
 'Aprendizado de máquina.',
 'Representação do conhecimento.',
 'Redes Neurais.']

#### Purificar os períodos e remover as stopwords;

In [183]:
emenda_filtrada = []
for periodo in periodos:
  stop_words = set(stopwords.words('portuguese')) 
  
  word_tokens = nltk.tokenize.word_tokenize(periodo) 
  
  filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
  filtered_sentence = [] 
  
  for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 
  print(filtered_sentence)
  emenda_filtrada.append(filtered_sentence) 

['Introdução', 'inteligência', 'artificial', ':', 'histórico', ',', 'conceitos', 'básicos', ',', 'áreas', 'aplicação', ',', 'tendências', '.']
['Métodos', 'resolução', 'problemas', '.']
['Técnicas', 'busca', ':', 'informação', '(', 'heurística', 'meta-heurística', '–', 'Algoritmos', 'Genéticos', ')', '.']
['Aprendizado', 'máquina', '.']
['Representação', 'conhecimento', '.']
['Redes', 'Neurais', '.']


#### Encontrar um texto na internet ou extrair de uma vídeo aula um texto

In [141]:
video_id = "1_c_MA1F-vU"
legendas_do_video = YouTubeTranscriptApi.get_transcript(video_id, languages=['pt', 'en'])
for i in range(len(legendas_do_video)):
  print(legendas_do_video[i]['text'])
  if i > 3 :
    break

[Música]
sejam bem vindos ao neurology até que eu
sou átila biólogo pesquisador e com das
minhas redes neurais para funcionar e eu
sou paulo silveira programador instrutor


####  Transformar o texto aula em períodos

In [143]:
#for i in range(len(legendas_do_video)):
  #legendas_do_video[i]['text'] = nltk.tokenize.sent_tokenize(str(legendas_do_video[i]['text']), language='portuguese')


#### Remover do texto aula as stopwords

In [144]:
for i in range(len(legendas_do_video)):
  stop_words = set(stopwords.words('portuguese')) 
  
  word_tokens = nltk.tokenize.word_tokenize(str(legendas_do_video[i]['text'])) 
  
  filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
  filtered_sentence = [] 
  
  for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 
  legendas_do_video[i]['text'] = (filtered_sentence) 

#### Analisar se o texto aula cobre o texto ementa.

In [189]:
for i in range(len(legendas_do_video)):
  print(legendas_do_video[i]['text'])
  if i > 3:
    break
print('==========================================')
for i in range(len(emenda_filtrada)):
  print(emenda_filtrada[i])

['[', 'Música', ']']
['bem', 'vindos', 'neurology']
['átila', 'biólogo', 'pesquisador']
['redes', 'neurais', 'funcionar']
['paulo', 'silveira', 'programador', 'instrutor']
['Introdução', 'inteligência', 'artificial', ':', 'histórico', ',', 'conceitos', 'básicos', ',', 'áreas', 'aplicação', ',', 'tendências', '.']
['Métodos', 'resolução', 'problemas', '.']
['Técnicas', 'busca', ':', 'informação', '(', 'heurística', 'meta-heurística', '–', 'Algoritmos', 'Genéticos', ')', '.']
['Aprendizado', 'máquina', '.']
['Representação', 'conhecimento', '.']
['Redes', 'Neurais', '.']
